In [13]:
"""
Fixing the error where didn't import the Dendrites for Spine Overlaying with this table
Was missing from the list so added it and reran so there will be spine data for dendrites


"""

"\nFixing the error where didn't import the Dendrites for Spine Overlaying with this table\nWas missing from the list so added it and reran so there will be spine data for dendrites\n\n\n"

In [14]:
import datajoint as dj
import numpy as np
import time

In [15]:
schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')
dj.config["display.limit"] = 30

In [16]:
@schema
class OverlayedSpineLabelOrphan(dj.Computed):
    definition = """
    # Segment labels with Spine labels overlayed (replacing) normal component labels for orphans.
    -> pinky.CompartmentOrphan
    ---
    vertices             : longblob                     
    triangles            : longblob  
    """

In [17]:
#pinky.OverlayedSpineLabelOrphan.drop()

In [18]:
#####Need to rerun after 2/20/18 when caught error#####
@schema
class OverlayedSpineLabelOrphan(dj.Computed):
    definition = OverlayedSpineLabelOrphan.describe()
    
    def make(self, key):
#         print(key['segment_id'])
#         start = time.time()
        
        #gets the labels from the proofreading
        corrected_labels = (pinky.CoarseLabelOrphan & key).fetch1()
        
        compartment_key = dict(segmentation=key['segmentation'], segment_id=key['segment_id'], decimation_ratio=0.35)
        #gets all of the components vertex and face indicies bt not the labels
        components = (pinky.CompartmentOrphan.ComponentOrphan & compartment_key
                      & [dict(compartment_type=compartment) for compartment in ["Apical", "Oblique", "Basal","Dendrite"]]).fetch()
        
        non_existence_count = 0
        component_spine_match = []
        for component in components:
            component_key = dict(compartment_key, compartment_type=component[3], component_index=component[4])
            try:
                #gets the final spine labels for that component
                component_labels = (pinky.ComponentLabelOrphan & component_key).fetch()
                #component_labels[component_labels==10] = 0
                #will get the number of heads 
                n_heads = list(zip(*component_labels))[11]
#                 min_n_heads = np.min(n_heads)
#                 index_supposed_best_fit = np.where(n_heads==min_n_heads)[0][0]#[-1]
#                 component_spine_match.append((component, list(component_labels[index_supposed_best_fit])[8:12]))

                #appends the component data along with the labels to a component spine matching list
                component_spine_match.append((component, list(component_labels[0])[7:11]))
            except:
                non_existence_count += 1
                pass
        print(f"{non_existence_count} components did not have spine data.")
        for match in component_spine_match:
            if not np.array_equal(list(match[0])[5:7], list(match[1])[:2]):
                print(match[0])
                print("Ruh-roh the sizes of the component and component_label array don't match.")
        
        corrected_vertex_labels = corrected_labels['vertices']
        corrected_triangle_labels = corrected_labels['triangles']

        new_complete_vertex_labels = corrected_vertex_labels.copy()
        for match in component_spine_match:
            vertex_indices = match[0][7]
            spine_vertex_labels = match[1][2].astype(np.uint8)
            overlayed_vertex_labels = np.array([corrected_vertex_labels[vertex_indices], spine_vertex_labels]).max(axis=0)
            new_complete_vertex_labels[vertex_indices] = overlayed_vertex_labels

        new_complete_triangle_labels = corrected_triangle_labels.copy()
        for match in component_spine_match:
            triangle_indices = match[0][8]
            spine_triangle_labels = match[1][3].astype(np.uint8)
            overlayed_triangle_labels = np.array([corrected_triangle_labels[triangle_indices], spine_triangle_labels]).max(axis=0)
            new_complete_triangle_labels[triangle_indices] = overlayed_triangle_labels
        
        self.insert1(dict(key,
                          vertices=new_complete_vertex_labels,
                          triangles=new_complete_triangle_labels))
        
#         print(time.time() - start, "\n")

# Segment labels with Spine labels overlayed (replacing) normal component labels for orphans.
-> pinky.CompartmentOrphan
---
vertices             : longblob                     
triangles            : longblob                     



In [19]:
start = time.time()
OverlayedSpineLabelOrphan.populate()
print(time.time() - start)

0 components did not have spine data.
0 components did not have spine data.
1 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
1 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components did not have spine data.
0 components

In [ ]:
OverlayedSpineLabelOrphan()

In [ ]:
"""
The trimesh method created a different number of vertices than the compartment component method --> so there was a mismatch

Solution: Need to rerun the spine using the Compartment Component solution


Fix is correctly working, the components that are said to not have spine data are those that are too small 
so without size threshold it is 2570 whereas if the size threshold is applied it is 2316

"""